# Graph search, Shortest path, and Data structure

## Generic graph search

- Find everything findable from a given start vertex
- Don't explore anything twice

```
genericAlgorithm(graph G, vertex s)
- initially s explored, all other vertices unexplored
- while possible
    - choose an edge (u, v) with u explored and v unexplored (if none, halt)
    - mark v explored
```

- Claim: at the end of algorithm, $v$ explored iff $G$ has path from $s$ to $v$
- Proof: (=>) easy proof by induction. (<=) by contradiction. suppose $G$ has path $P$ from $s$ to $v$ but $v$ unexplored at the end of algorithm. then there exists an edge $(u,x)$ in $P$ such that $u$ explored and $v$ unexplored. but then algorithm would not have terminated  

## BFS

- Explore ndoes in "layers"
- Can compute shortest path
- Can compute connected components of undirected graph
- $O(n+m)$ using queue

```
BFS(graph G, start vertex s)
- [all node initially unexplored]
- mark s as explored
- let Q = queue initialized with s
- while Q is not empty:
    - remove first node of Q, call it v
    - for each edge (v, w)
        - if w unexplored
            - mark w as explored
            - add w to Q (at the end)
```

- Claim: at the end of BGS, $v$ explored iff $G$ has path from $s$ to $v$
- Proof: special case of generic algorithm
- Claim: run time is $O(n+m)$
- Proof: inspection of code

### Application - Shortest path

- Compute dist(v), the fewest number of edges on path from $s$ to $v$
- Assumption: every edge has length of 1 
- Extra code to BFS

```
- initialize dist(v): 0 if v=s, large number if v != s
- when considering edge (v,w)
    - if w unexplored, then set dist(w) = dist(v) + 1 
```

### Application - Undirected connectivity

- Let $G(V,E)$ undirected graph
- Connected component = $pieces of G$
- Compute all connected components

```
- initalize: all nodes unexplored
- assume labelled 1 to n
- for i = 1 to n
    - if i not explored # in some previsou BFS
        - BFS(G, i) # discovers precisely i's connected component
```

## DFS

- Expllore aggressively, only backtrack when necessary
- Can compute topological ordering of directed acyclic graph
- Can compute strongly connected components of directed graph
- $O(n+m)$ using stack

```
DFS(graph G, start vertex s)
- mark s as explored
- for every edge (s, v)
    - if v is unexplored
        - DFS(G, v)
```   
   
### Application - Topological ordering

- Label $f$ on nodes of $G$ such that
    - $f(v)$'s are the set {1,2,...,n}
    - $(u,v) \in G$ => $f(n) \le f(v)$
- Let $v$ a sink vertex of $G$ (every directed graph has a sink vertex)
- Set $f(v) = n$
- Recurse on $G - \{v\}$
- If $G$ has directed cycle, then there is no topological ordering
- If $G$ does not have directed cycle, then computes topological ordering in $O(m+n)$

```
DFS_loop(graph G) 
- mark all nodes unexplored
- current_label = n # keep track of ordering
- for each vertext v in G
    - if v not explored
        - DFS(G,v)

DFS(graph G, start vertex s)
- mark s as explored
- for every edge (s, v)
    - if v is unexplored
        - mark v explored
        - DFS (G, v)
- set f(s) = current_label
- current_label--
```

Correctness
- Need to show that if $(u,v)$ is on edge, then $f(u) \lt f(v)$
- Case #1: $u$ is visited by DFS before $v$. then recursive call corresponding to $v$ finishes before $u$, $f(v) \gt f(u)$
- Case #2: $v$ is visited by DFS before $u$. then recursive call corresponding to $v$ finishes before $u$ even starts, $f(v) \gt f(u)$

### Application - Strongly connected components

- There exist path $u \rightarrow v$ and $v \rightarrow u$ in graph G

Kosaraju's two pass algorithm
- Compute SCC in $O(m+n)$
- Let $G^{'} = G$ with all arcs reversed
- Run DFS_loop on $G^{'}$ (compute magical ordering of nodes)
- Run DFS_loop on $G$ (compute strongly connected component one by one)

```
DFS_loop(graph G)
- global variable t=0 # number of nodes processed so far
- global variable s=null # current source vertex
- assumes nodes labelled 1 to n
- for i = n to 1
    - if i not explored 
        - s = i
        - DFS(G, i)
        
DFS(graph G, node i)
- mark i as explored
- set leader(i) = node s
- for each arc (i,j) in G
    - if j not explored
        - DFS(G, j)
- t++
- set f(i) = t # ith finishing time
```

Correctness
- Claim: SCC of a directed graph $G$ induces an acyclic "meta-graph"
- Notice SCCs of original graph $G$ and its reversal $G^{'}$ are exactly the same
- Lemma: consider two "adjacent" SCCs in $G$. let $f(v)$ = finishing times of DFS_loop in $G^{'}$. then $max_{v \in C_{1}} f(v) \lt max_{v \in C_{2}} f(v)$
- Corollary: maximum f-value of $G$ must lie in a "sink SCC"
- By corollary, 2nd DFS_loop begins somewhere in a sink SCC $C^{*}$ 
    - First call to DFS discovers $C^{*}$, nothing else
    - Rest of DFS_loop like recursing on $G$ with $C^{*}$ deleted
    - Successive calls to DFS(G,i) "peel off" the SCCs one by one

In [ ]:
import math
import random
import collections
import sys
import threading

In [ ]:
def DFS_ordering(graph, node, ordering, explored_ordering):
    """
    Perform depth first search on graph starting at the given node in order to compute 'magical ordering'

    Args:
    graph (dictionary) -- adjacency represeantaion of graph
    node (integer) -- node to start searching
    ordering (dictionary) -- represents finishing order of each node
    explored_ordering (set) -- stores already explored nodes

    Returns:
    None
    """

    explored_ordering.add(node)
    if node in graph: # if node has outgoing edge(s)
        for vertex in graph[node]: # loop through outgoing vertices of given node
            if vertex not in explored_ordering:
                DFS_ordering(graph, vertex, ordering, explored_ordering)
    
    ordering[node] = len(ordering) + 1

In [ ]:
def DFS_loop_ordering(graph, max_integer, ordering, explored_ordering):
    """
    Perform depth first search for all nodes from max_integer to 1 in order to compute 'magical ordering'

    Args:
    graph (dictionary) -- adjacency represeantaion of graph
    max_integer (integer) -- number of nodes to perform searching
    ordering (dictionary) -- represents finishing order of each node
    explored_ordering (set) -- stores already explored nodes

    Returns:
    None
    """

    i = max_integer
    while i > 0:
        if i not in explored_ordering:
            DFS_ordering(graph, i, ordering, explored_ordering)
        i = i - 1

In [ ]:
def DFS_computing(graph, node, leader, explored_computing, s):
    """
    Perform depth first search on graph starting at the given node in order to compute 'leaders' for strongly connected components

    Args:
    graph (dictionary) -- adjacency represeantaion of graph
    node (integer) -- node to start searching
    leader (list) -- leader nodes of strongly connected components
    explored_computing (set) -- stores already explored nodes
    s (list) -- leader node of a strongly connected component

    Returns:
    None
    """

    explored_computing.add(node)
    leader.append(s[0])
    if node in graph: # if node has outgoing edge(s)
        for vertex in graph[node]: # loop through outgoing vertices of given node
            if vertex not in explored_computing:
                DFS_computing(graph, vertex, leader, explored_computing, s)

In [ ]:
def DFS_loop_computing(graph, max_integer, leader, explored_computing, s):
    """
    Perform depth first search for all nodes from max_integer to 1 in order to compute 'leaders' for strongly connected components

    Args:
    graph (dictionary) -- adjacency represeantaion of graph
    max_integer (integer) -- number of nodes to perform searching
    leader (list) -- leader nodes of strongly connected components
    explored_computing (set) -- stores already explored nodes
    s (list) -- leader node of a strongly connected component

    Returns:
    None
    """

    i = max_integer
    s[0] = 0
    while i > 0:
        if i not in explored_computing:
            s[0] = i
            DFS_computing(graph, i, leader, explored_computing, s)
        i = i - 1

In [ ]:
def get_next(graph, node):
    """
    Get all outgoing vertices from given node

    Args:
    graph (dictionary) -- adjacency represeantaion of graph
    node (integer) -- node to find outgoing vertices

    Returns:
    vertices (set) -- all outgoing vertices from given node
    """

    vertices = set()
    for arc in graph:
        if arc[0] == node:
            vertices.add(arc[1])
            
    return vertices

In [ ]:
def compute_max(graph):
    """
    Computes maximum number from the given graph

    Args:
    graph (list of lists) -- adjacency representation of graph 

    Returns:
    max_num (integer) -- maximum number from given graph
    """

    temp_list = []
    for edge in graph:
        temp_list.append(max(edge[0], edge[1]))

    max_num = max(temp_list)
    return max_num

In [ ]:
def open_graph_as_list(file_path):
    """
    Imports a file and stored data into a list of lists

    Args:
    file_path (string) -- location of file

    Returns:
    graph (list of lists) -- adjacency representation of graph 
    """

    graph = []

    with open(file_path, 'r') as line:
        array = line.read().split("\n")
        for subarray in array:
            graph.append(subarray.split(" "))

    for arc in graph:
        arc[0] = int(arc[0])
        arc[1] = int(arc[1])

    return graph

In [ ]:
def convert_graph_to_dict(graph_list):
    """
    Imports a file and stored data into a dictionary

    Args:
    file_path (string) -- location of file

    Returns:
    graph (dictionary) -- adjacency represeantaion of graph
    """

    graph_dict = {}

    for arc in graph_list:
        key = int(arc[0])
        value = int(arc[1])
        if key not in graph_dict:
            graph_dict[key] = set()
            graph_dict[key].add(value)
        else:
            graph_dict[key].add(value)

    return graph_dict

In [ ]:
def strongly_connected_component():
    """
    Compute strongly connected components

    Args:
    None

    Returns:
    None
    """
    
    ordering = {} # dictionary to store magical ordering. dictionary is to have O(1) for loopkup
    explored_ordering = set() # set to store explored nodes. set is to have O(1) for loopkup
    
    leader = []
    explored_computing = set()
    s = []
    s.append(-1) # leaders in second path

    # Convert graph (list) to graph(dictionary) to have O(1) for looking up outgoing vertices from given node
    graph_dict = convert_graph_to_dict(graph_list) 
    
    # Data is provided such that nodes are labelled from 1 to max_integer
    max_integer = compute_max(graph_list) 
    
    # Compute the magical ordering
    DFS_loop_ordering(graph_dict, max_integer, ordering, explored_ordering) 
    
    # Reverse direction of graph (~ 10 seconds)
    for edge in graph_list:
        tmp = edge[0]
        edge[0] = edge[1]
        edge[1] = tmp

    # Change nodes based on magical ordering
    for i in range(0, len(graph_list)):
        graph_list[i][0] = ordering.get(graph_list[i][0])
        graph_list[i][1] = ordering.get(graph_list[i][1])

    graph_dict = convert_graph_to_dict(graph_list)
    DFS_loop_computing(graph_dict, max_integer, leader, explored_computing, s)

    # Show the result
    counter = collections.Counter(leader)
    result = ""
    for tuple_item in counter.most_common(5):
        result += str(tuple_item[1])
        result += ","
    print(result[:-1])
    return result[:-1]

In [ ]:
graph_list = open_graph_as_list("data/strongly-connected-component1.txt")
assert(strongly_connected_component() == "3,3,3")

graph_list = open_graph_as_list("data/strongly-connected-component2.txt")
assert(strongly_connected_component() == "3,3,2")

graph_list = open_graph_as_list("data/strongly-connected-component3.txt")
assert(strongly_connected_component() == "3,3,1,1")

graph_list = open_graph_as_list("data/strongly-connected-component4.txt")
assert(strongly_connected_component() == "7,1")

graph_list = open_graph_as_list("data/strongly-connected-component5.txt")
assert(strongly_connected_component() == "6,3,2,1")

graph_list = open_graph_as_list("data/strongly-connected-component.txt")
sys.setrecursionlimit(8000000)
threading.stack_size(67108864)
thread = threading.Thread(target=strongly_connected_component)
thread.start()

## Shortest path (Dijkstra's Algorithm)

Single-source shortest path
- Input: directed graph $G = (V,E)$
    - Each edge has non-negative length $l_{e}$
    - Source vertex $s$
- Output: for each $v \in V$, compute $L(v)$ = length of a shortest $s$-$v$ path in $G$

BFS computes shortest paths in linear time if $l_{e} = 1$ for every edge $e$. We can replace each edge $e$ by a number of $l_{e}$ with unit length, but it blows up graph too much

### Implementation (run in $O(nm)$)
- $X = \{s\}$ # vertices processed so far
- $A[s] = 0$ # computed shortest path distances
- $B[s] = null$ # computed shortest path (actial path like a->b->c)
- While $X$ != $V$ # assume there are two sets $X$ and $V-X$ 
    - Among all edges $(v,w) \in E$ with $v \in X$, $w \notin X$, pick the one that minimizes $A[v]$ + $l_{vw}$ # call it $(v^{*}, w^{*})$
    - Add $w^{*}$ to $X$
    - Set $A[w^{*}]$ = $A[v^{*}] + l_{v^{*}w^{*}}$
    - Set $B[w^{*}]$ = $B[v^{*}] + (v^{*}, w^{*})$
    
Why can't add a large number to $l_{e}$ if there are edges with negative length? because it does not preserve the shortest path

### Correctness
- Claim: for every directed graph with non-negative edge length, Dijkstra's algorithm correctly computes all shortest path distances. In other words, $A[v] = L(v), \forall{v} \in V$ where $A[v]$ = what algorithm computes and $L(v)$ = true shortest distance
- Proof
    - Base case $A[s] = L[s] = 0$
    - Assume $A[v] = L[v]$ and $B[v]$ = true shortest $s$-$v$ path in $G$, for all $v$ already in $X$
    - Then, 
        - We pick an edge $(v^{*}, w^{*})$ and we add $w^{*}$ to $X$. 
        - We set $B[w^{*}]$ = $B[v^{*}] + (v^{*}, w^{*})$ = an $s$->$w^{*}$ path with length $L(v^{*}) + l_{v^{*}w^{*}}$
        - Beed to show every $s$-$w^{*}$ path has length $\ge L(v^{*}) + l_{v^{*}w^{*}}$  
            - Let $P$ = any $s$-$w^{*}$ path
            - Then $s \rightarrow y \in X \rightarrow z \notin X \rightarrow w^{*} \notin X$ 
            - Then $A[v^{*}] + l_{v^{*}w^{*}} \le A[y] + l_{yz} \le$ length of $P$
            
### Heap
- Perfectly balanced tree
- At every node, key $\le$ children's keys
- Extract-Min by swapping up last leaf, bubbling down
- Insert via bubbling up

Two invariants
1. Elements in heap = vertices of $V-X$
2. For $v \notin X$, $key[v]$ = smallest Dijkstra greedy score of an edge $(u,v)$ in $E$ with $v \in X$

By this, Extract-Min yields correct vertex $w^{*}$ to add to $X$ next (and we set $A[w^{*}]$ to $key[w^{*}]$)

To maintain invariant #2, when $w$ extracted from heap (added to $X$)
- For each edge $(w,v)$ in $E$
    - If $v$ in $V-X$ (in heap)
        - Delete $v$ from heap
        - Re-compute $key[v]$ = $min\{key[v], A[w]+l_{wv}\}$
        - Re-insert $v$ into heap
        
Runtime analysis
- $(n-1)$ Extract-Mins
- Each edge $(v,w)$ triggers at most one delete/insert combo (if $v$ added to $X$ first)
- Number of heap operations in $O(n+m) = O(m)$ (since graph is weakly connected)
- Runtime = $O(mlogn)$

In [ ]:
def open_graph(file_path):    
    """
    Imports a file and stored data into a list of lists
    
    Args:
    file_path (string) -- location of file
    
    Returns:
    graph (list of lists) -- adjacency representation of graph 
    """
    
    graph = []
    
    with open(file_path, 'r') as line:
        array = line.read().split("\n")
        for subarray in array:
            graph.append(subarray.split("\t"))
    
    for i in range(0, len(graph)):
        del graph[i][-1]
    del graph[-1]
    
    return graph

In [ ]:
def get_next(graph, source_vertex):
    """
    Get all possible outgoing vertices from a given vertex
    
    Args:
    graph (list of lists) -- adjacency representation of graph 
    source_vertex (string) -- given vertex
    
    Returns:
    vertices (list) -- all possible outgoing vertices
    """
    
    vertices = []
    
    for elem in graph:
        if elem[0] == source_vertex:
            for i in range(1, len(elem)):
                vertices.append(elem[i].split(",")[0])
    
    return vertices

In [ ]:
def get_all_outgoing_path(graph, X):
    """
    For all nodes in X, find outgoing vertices that are not in X
    
    Args: 
    graph (list of lists) -- adjacency representation of graph 
    X (list) -- all explored vertices
    
    Returns:
    candidates (list) -- all outgoing vertices, that are not in X, for all vertices in X
    """
    
    candidates = []
    
    for vertex1 in X:
        outgoing = get_next(graph, vertex1)
        for vertex2 in outgoing:
            if vertex2 not in X:
                candidates.append((vertex1, vertex2))
            
    return candidates

In [ ]:
def get_candidates_for_source_vertex(candidates, destination_vertex):
    """
    Find all source vertices whose outgoing vertices are the final destination vertex
    
    Args:
    candidates (list) -- all outgoing vertices, that are not in X, for all vertices in X
    destination_vertex (string) -- final destination vertex
    
    Returns:
    candidates_for_source_vertex (list) -- all vertices whose outgoing vertices are the final destination vertex
    """

    candidates_for_source_vertex = []
    
    for edge in candidates:
        if edge[1] == destination_vertex:
            candidates_for_source_vertex.append(edge[0])
    return candidates_for_source_vertex

In [ ]:
def get_cost(graph, source_vertex, destination_vertex):
    """
    Find cost between vertices
    
    Args:
    graph (list of lists) -- adjacency representation of graph 
    source_vertex (string) -- a given vertex
    destination_vertex (string) -- an outgoing vertex for given vertex
    
    Returns:
    cost (integer) -- cost between given and outgoing vertices
    """

    cost = -1
    for elem in graph:
        if elem[0] == source_vertex:
            for i in range(1, len(elem)):
                if elem[i].split(",")[0] == destination_vertex:
                    cost = elem[i].split(",")[1]
    return int(cost)

In [ ]:
def pick_minimum(graph, A, X, candidates):    
    """
    For all outgoing vertices from X, find an outgoing vertex that would incur the minimum cost. Then update X and A
    
    Args:
    graph (list of lists) -- adjacency representation of graph 
    A (dictionary) -- key represents a vertex and value represents the cost from the initial source vertex to this vertex
    X (list) -- all explored vertices
    candidates (list) -- all outgoing vertices, that are not in X, for all vertices in X
    
    Returns:
    None
    """
    
    minimum_distance = 1000000
    minimum_vertex2 = ""
    
    for candidate in candidates:
        vertex1 = candidate[0] # this vertex is in X
        vertex2 = candidate[1] # this vertex is not in X
        
        for elem in graph:
            if elem[0] == vertex1:
                for i in range(1, len(elem)):
                    if elem[i].split(",")[0] == vertex2:
                        index_of_vertex1 = X.index(vertex1)
                        if A[vertex1] + int(elem[i].split(",")[1]) < minimum_distance:
                            minimum_distance = A[vertex1] + int(elem[i].split(",")[1])
                            minimum_vertex2 = elem[i].split(",")[0]
    
    A[minimum_vertex2] = minimum_distance
    X.append(minimum_vertex2) 

In [ ]:
def shortest_path(graph, source_vertex, destination_vertex, A, X, init=False):
    """
    Compute the shortest path algorithm on a directed graph
    
    Args:
    graph (list of lists) -- adjacency representation of graph 
    source_vertex (string) -- initial vertex to start from
    destination_vertex (string) -- final vertex to end
    A (dictionary) -- key represents a vertex and value represent the cost from the initial source vertex to that vertex
    X (list) -- all explored vertices
    init (string) -- flag to determin whether to initialize A and X
    
    Returns:
    A[destination_vertex] (integer) -- cost from the source vertex to the final destination vertex
    """

    if init:
        A[source_vertex] = 0 # distance from source_vertex to itself is 0
        X.append(source_vertex) # source_vertex is the first vertext in the set

    candidates = get_all_outgoing_path(graph, X)
    if destination_vertex not in X and len(X) < len(graph):
        pick_minimum(graph, A, X, candidates)
        shortest_path(graph, X[-1], destination_vertex, A, X)
        
    # If there is no path between the original source and destination
    if destination_vertex not in X: 
        A[destination_vertex] = 1000000

    return A[destination_vertex]

In [ ]:
graph = open_graph("data/shortest-path.txt")
assert(shortest_path(graph, "1", "7", {}, [], True) == 2599)
assert(shortest_path(graph, "1", "37", {}, [], True) == 2610)
assert(shortest_path(graph, "1", "59", {}, [], True) == 2947)
assert(shortest_path(graph, "1", "82", {}, [], True) == 2052)
assert(shortest_path(graph, "1", "99", {}, [], True) == 2367)
assert(shortest_path(graph, "1", "115", {}, [], True) == 2399)
assert(shortest_path(graph, "1", "133", {}, [], True) == 2029)
assert(shortest_path(graph, "1", "165", {}, [], True) == 2442)
assert(shortest_path(graph, "1", "188", {}, [], True) == 2505)


assert(shortest_path(graph, "1", "197", {}, [], True) == 3068)

## Data Structure

- Organize data so that it can be accessed quickly and usefully
- Ex: list, stack, queue, heap, search tree, hashtable, bloom filter, union-find, etc
- Choose the "minimal" data structure that supports all the operations that you need

## Heap (a.k.a priority queue)

- A container for objects that have keys
- Ex: employer records, network edges, events, etc

Operatons
- Insert: add a new object to heap $O(nlogn)$
- Extract: remove an object with minimum key valye $O(nlogn)$
- Heapify: $n$ batched inserts $O(n)$
- Delete: $O(nlogn)$

Property
- Think of a heap as a tree: rooted, binary, as complete as possible
- At every node $x$, $key[x] \le$ all keys in $x$'s children
- Object at root must have minimum key value

Array implementation
- Put node in the tree into array layer by layer
- Parent$(i)$ = $i/2$ if $i$ is even, floor$(i/2)$ if $i$ is odd
- Children$(i)$ = $2i$ and $2i + 1$

```
Insert(key k)
- stick k at the end of last level
- bubble-up k until k's parent <= k
```

```
Extract-Min
- delete root
- move last node to new root
- bubble-down k until k's parent <= k
```

### Application - heapsort

- Insert all $n$ array elements into a heap  
- Extract-min to pluck out elements in sorted order 
- $O(nlogn)$

### Application - event manager

- Objects: event records
- Key: time event scheduled to occur
- Extract-min: yields the next scheduled event

### Application - median maintenance

- Given a sequence $x_{1} \dots x_{n}$ of numbers coming one at a time, find median of $\{x_{1} \dots x_{i}\}$ at each time step $i$
- $H_{low}$: supports extract-min
- $H_{high}$: supports extract-max
- Maintain invariant that $i/2$ smallest element in $H_{low}$ (or $i/2$ largest element in $H_{high}$)

### Sorted array

- Search: $O(logn)$
- Select(given order statistic $i$): $O(1)$
- Min/Max: $O(1)$
- Pred/Succ (given pointer to a key): $O(1)$
- Rank (number of kyes less than or equal to a given value): $O(logn)$
- Output in sorted order: $O(n)$
- Insert/Delete: $O(n)$

### Balanced search tree (sorted array with fast insert & delete)

- Search: $O(logn)$
- Select: $O(logn)$
- Min/Max: $O(logn)$
- Pred/Succ: $O(logn)$
- Rank: $O(logn)$
- Output in sorted order: $O(n)$
- Insert/Delete: $O(logn)$

### Binary search tree 

- Exactly one node per key
- Each node has
    - Left child pointer
    - Right child pointer
    - Parent
- All nodes left on node $X$ are less than $X$
- All nodes right on node $X$ are greater than $X$
- Many possible trees for a set of keys
- Height could be anywhere from $log_{2}n$ to $n$
- Generally operations are $O(height)$

```
Search(key k)
- start at the root
- traverse left (if k < key at current node) or right (if k > key at current node) child pointers as needed
- return node with key k or NULL, as appropriate
```

```
Insert(key k)
- start at the root
- do search (which will return NULL)
- rewire final NULL pointer to point to new node with key k
```

```
Min/Max
- start at the root
- follow left (min case) or right (max case) until the bottom (return last key found)
```

```
Pred(key k)
- easy case: if k's left subtree nonempty, return max key in left subtree
- otherwise: follow parent pointers until you get to a key less than k
```

```
Inorder traversal
- to print out keys in increasing order
- let r = root, Tr = right subtree, Tl = left subtree
- recurse on Tl
    - by recursion, prints out keys of TL in increasing order
- print out r's key
- recurse on Tr
    - by recursion, prints out keys of TR in increasing order
```

```
Delete(key k)
- search for k
- if k has no children 
    - delete k
- k has one child
    - delete k, and put child under k's parent
- k has two children 
    - compute k's predecessor l
        - for example, traverse k's (non-NULL) left child pointer, then right child pointers until no longer possible
    - swap k and l
    - delete k
```

```
Select(order statistic i )
- store a little bit of extra info at each tree node about the tree itself
- start at root x, with children y and z
- let a = size(y) # a = 0 if x has no left child
- if a = i-1
    - return x's key
- if a >= i
    - recurse to compute ith order statistic on new root y
- if a < i-1
    - recurse to compute (i-a-1)th order statistic on new root z
```

### Balanced search tree 

- Ensure that heights are $O(logn)$ so that search/insert/delete/min/max/pred/succ will run in $O(logn)$
- Ex: red-black tree, AVL, splay tree, B tree

### Red-Black invariants

1. Each node red or black
2. Root is black
3. No 2 reds in a row (red node => only black children)
4. Every root-NULL path (unsuccessful search) has the same number of black nodes

### Height guarantee

- Claim: every red-black tree with $n$ nodes has height $\le 2log_{2}(n+1)$ 
- Proof: 
    - If every root-NULL path has $\ge k$ nodes, then tree includes (at the top) a perfectly balanced search tree of depth $k-1$
    - Thus, size $n$ of the tree must be at least $2^{k}-1$
    - Then, $k \le log_{2}(n+1)$
    - Then, there is a root-NULL path with at most $log_{2}(n+1)$ black nodes
    - By invariant 4, every root-NULL path has $\le log_{2}(n+1)$ black nodes
    - By invariant 3, every root-NULL path has $\le log_{2}(n+1)$ total nodes
    
### Rotation

- Locally rebalance subtrees at a node in $O(1)$ time
- Left rotation
- Right rotation

### Insertion in a Red-Black tree

- Proceed as in a normal binary search tree, then recolor and (or perform rotation) until invariants are restored

```
Insert(x)
- insert x as usual (makes x a leaf)
- try coloring x red
- if x's parent y is black, done
- else y is red, then y has a black parent w
```

Case #1: the other child $z$ of $x$'s grand parent $w$ is also red
- Recolor $y,z$ black and $w$ red (this does not break invariant 4)
- Either restores invariant 3 or propagate the double red upward
- Can only happen $O(logn)$ times (if you reach the root, recolor it black => preserves invariant 4)

Case #2: let $x,y$ be the current double-red, $x$ the deeper node. let $w$ = $y$'s grand parent. suppose $w$'s other child (which is not equal to $y$) is NULL or is a black node $z$
- Can eliminate double-red (all invariants satisfied) in $O(1)$ time via 2-3 rotations + recolorings

In [ ]:
def open_file(file_path):
    """
    Read file line by line
    
    Args:
    file_path (string) -- location of file to be read
    
    Returns:
    array (list) -- contains integers
    """
    
    array = []
    
    with open(file_path, 'r') as line:
        array = line.read().split("\n")
        
    for i in range(0, len(array)):
        array[i] = int(array[i])
        
    return array

In [ ]:
def adjust_two_heaps(low_heap, high_heap):
    """
    Adjusts two heaps such that lower half of set is in low_heap and upper half is in high_heap
    
    Args:
    low_heap (list) -- heap that holds lower half of data
    high_heap (list) -- heap that holds upper half of data
    
    Returns:
    None
    """
    
    max_low_heap = max(low_heap)
    min_high_heap = min(high_heap)
    
    if max_low_heap > min_high_heap:
        min_high_heap_index = high_heap.index(min_high_heap)
        del high_heap[min_high_heap_index]
        high_heap.append(max_low_heap)
        
        max_low_heap_index = low_heap.index(max_low_heap)
        del low_heap[max_low_heap_index]
        low_heap.append(min_high_heap)
        
        adjust_two_heaps(low_heap, high_heap)

In [ ]:
def compute_running_median(array):
    """
    Computes running median of an input array
    
    Args:
    array (list) -- contains numbers
    
    Returns:
    median (list) -- contains running medians of an input array
    """
    
    low_heap = []
    high_heap = []
    median = []
    
    for i in array:
        if len(low_heap) > len(high_heap):
            high_heap.append(i)
        else:
            low_heap.append(i)

        if len(high_heap) > 0 and len(low_heap) > 0:
            adjust_two_heaps(low_heap, high_heap)

        if len(high_heap) == 0:
            median.append(low_heap[0])

        if len(low_heap) == 0:
            median.append(high_heap[0])

        if len(high_heap) > 0 and len(low_heap) > 0:
            median.append(max(low_heap))
        
    return median

In [ ]:
array = open_file("data/median-maintenance.txt")
median = compute_running_median(array)
assert(sum(median) % 10000 == 1213)

## Hash table

- Maintain (or evoling) set of stuff
- Ex: transactions, people + associated data, IP addresses
- Really a dictionary (w/o ordering elements)
- Insert & Delete & Lookup: $O(1)$

### Application - de-duplication

- Given a "stream" of objects, remove duplications
- When new object $x$ arrives
    - Lookup $x$ in hash table $H$
    - If not found, insert $x$ into $H$
    
### Application - 2-SUM problem

- Input: unsorted array $A$ of $n$ integers and target sum $t$
- Goal: determine whether or not there are two numbers $x,y$ in $A$ such that $x + y = t$
- Exhaustive search: $\theta(n^{2})$
- Sort $A$, then for each $x$ in $A$, look for $t-x$ in $A$ via binary search: $\theta(nlogn)$
- Insert elements of $A$ into hash table $H$, then for each $x$ in $A$, lookup $t-x$: $\theta(n)$

### High level idea

- Setup: universe $U$
- Goal: want to maintain evolving set $S \subseteq U$
- Pick $n$ = number of "buckets"
- Choose a hash function $h: U -> \{0,1,2 \dots n-1\}$
- Use array $A$ of length $n$, store $x$ in $A[h(x)]$

### Resolving collisions

- Collision: distinct $x,y \in U$ such that $h(x) = h(y)$ 

Solution #1: (separate) chaining
- Keep linked list in each bucket
- Given a key/object $x$, perform insert/delete/lookup in the list in $A[h(x)]$ where $A$ = linked list for $x$ and $h(x)$ = bucket for $x$
 
Solution #2: open addressing (only one object per bucket)
- Hash function now specifies probe sequence $h_{1}(x), h_{2}(x) \dots$ (keep trying till finding an open slot)
- Ex: linear probing (look consecutively), double hashing
 
###  Good hash function
- Spread data out
- Easy to store
- Fast to evaluate

### Universal hash function

- The load of a hash table $\alpha$ = (number of objects in hash table) / (number of buckets of hash table)
    - $\alpha = O(1)$ is necessary condition for operations to run in constant time
    - With open addressing, need $\alpha << 1$
- Good hash table performance
    - Need to control load
    - Need a good hash function
    
Problem
- Ideally we want hash function that guarantees to spread every data set out evenly, however for every hash function, there is a pathological data set

Solution
- Use a cryptographic hash function (ex. SHA-2)
    - Infeasible to reverse engineer a pathological data set 
- Use randomization
    - Design a family $H$ of hash functions such that for all data sets $S$, "almost all" functions $h \in H$ spread $S$ out "pretty evenly"
    
Definition
- Let $H$ be a set of hash functions from $U$ to $\{0,1,2 \dots n-1\}$
- $H$ is universal iff for all $x,y$ in $U$ (with $x != y$)
- $Pr_{h \in H}[h(x) = h(y)] \le \dfrac{1}{n}$, $n$ = number of buckets, $h$ is chosen uniformly at random from $H$  

### Example - hashing IP addresses

- Let $U$ = IP addresses of the form $(x_{1}, x_{2}, x_{3}, x_{4})$ with each $x_{i} \in \{0,1,2 \dots 255\}$
- Let $n$ = a prime (ex. small multiple of number of objects in hash table)
- Define one hash function $h_{a}$ per 4-tuple $a = (a_{1}, a_{2}, a_{3}, a_{4})$ with each $a_{i} \in \{0,1,2 \dots n-1\}$
- $h_{a}$: IP addresses -> buckets by
    - $h_{a}(x_{1}, x_{2}, x_{3}, x_{4}) = (a_{1}x_{1} + a_{2}x_{2} + a_{3}x_{3} + a_{4}x_{4})$ mod $n$
    - $H = \{h_{a} | a_{1}, a_{2}, a_{3}, a_{4} \in \{0,1,2 \dots n-1\} \}$
- Claim: this family is universal
- Proof:
    - Consider distinct IP addresses $(x_{1}, x_{2}, x_{3}, x_{4}), (y_{1}, y_{2}, y_{3}, y_{4})$
    - Assume $x_{4} != y_{4}$
    - Collision <=> $a_{1}x_{1} + a_{2} + x_{2} + a_{3} + x_{3} + a_{4}x_{4} = a_{1}y_{1} + a_{2} + y_{2} + a_{3} + y_{3} + a_{4} + y_{4}$ <=> $a_{4}(x_{4} - y_{4}) = \displaystyle\sum_{i=1}^{3}a_{i}(y_{i} - x_{i})$ mod $n$
    - With $a_{1}, a_{2}, a_{3}$ fixed arbitrarily, how many choices of $a_{4}$ satisfy
        - $a_{4}(x_{4} - y_{4}) = \displaystyle\sum_{i=1}^{3}a_{i}(y_{i} - x_{i})$ mod $n$
    - Notice that LHS is equally likely to be any of $\{0,1,2 \dots n-1\}$ ($x_{4} != y_{4}$, $n$ is prime, $a_{4}$ uniform at random)
    
### Chaining: constant-time guarantee

- Scenario: hash table implemented with chaining. hash function $h$ chosen uniformly at random from universal family $H$
- Theorem: all operations run in $O(1)$ for every data set $S$
- Caveats:
    - In expectation over the random choice of the hash function $h$
    - Assumes $|S| = O(n)$ (ex. load $\alpha = \dfrac{|S|}{n} = O(1)$)
    - Assumes $O(1)$ to evaluate hash function
- Proof: only analyze an unsuccessful lookup because other operations are faster
    - Let $S$ = data set with $|S| = O(n)$
    - Consider lookup for $X \notin S$
    - Runtime: $O(1) + O$(list length in$A[h(x)])$ (compute $h(x)$ + traverse list)
    - Let $L$ = list length in $A[h(x)]$
    - For $y \in S$ (so $y != x$) define $z_{y} = 1$ if $h(y) = h(x)$, $0$ otherwise
    - Note $L = \displaystyle\sum_{y \in S}A_{y}$
    - So $E[L] = \displaystyle\sum_{y \in S}E[z_{y}]$
    - $E[z_{y}] = 0 * Pr[z_{y} = 0] + 1 * Pr[z_{y} = 1] = Pr[h(y) = h(x)]$
    - $E[L] = \displaystyle\sum_{y \in S}E[z_{y}] = \displaystyle\sum_{y \in S}Pr[h(y) = h(x)] \le \displaystyle\sum_{y \in S}\dfrac{1}{n}$ (because $H$ is universal and by definition of universal hash function) = $\dfrac{|S|}{n} = \alpha = O(1)$
    
### Open addressing

- One object per slot, hash function produces a probe sequence for each possible key $x$
- Difficult to analyze rigorousely
- Heuristic assumption: all $n!$ probe sequences equally
- Claim: under heuristic assumption, expected insertion time is $\dfrac{1}{1-\alpha}$, where $\alpha$ = load
- Proof
    - A random probe finds an empty slot with probability $1-\alpha$
    - Thus, insertion time is approximately equals to the number $N$ of coin flips to get "heads", where $Pr[$"heads"$] = 1-\alpha$
    - Note $E[N]$ = (1st coin flip) + (probability of tails)(expected number of further coin flip needed) = $1 + \alpha E[N]$
    - Thus $E[N] = \dfrac{1}{1-\alpha}$
    
### Linear probing
- Heuristic assumption is completely false
- Assume instead: initial probes uniform at random independent for different keys
- Theorem: expected insertion time = $\dfrac{1}{(1-\alpha)^{2}}$

### Bloom filters

- Fast insert and lookup
- Compare to hash table
    - More space efficient (pro)
    - Can't store an associated object (con)
    - No delete (con)
    - Small false positive probability (con) 
        - Might say $x$ has been inserted although it hasn't
- No false negative (if $x$ was inserted, lookup(x) guaranteed to succeed) 
- False positive if all $k$ $h_{i}(x)$'s are already set to $1$ by other insertions 
        
Applications
- Early spellcheckers
- List of forbidden passwords
- Network routers (limited memory, so need to be super fast)

Ingredients
- Array of $n$ bits ($\dfrac{n}{|S|}$ = number of bits per object in data set $S$)
- $k$ hash functions $h_{1} \dots h_{k}$, where $k$ is a small constant

Insert(x)
- For $i = 1,2 \dots k$ (whether or not bit already set to 1)
    - Set $A[h_{i}(x)] = 1$
    
Lookup(x)
- Return True <=> $A[h_{i}(x)] = 1$ for every $i = 1,2 \dots k$ 

### Heuristic analysis

- Intuition: should be a trade-off between space and error (false positive) probability
- Assume: all $h_{i}(x)$'s uniformly random and independent (across different $i$'s and $x$'s)
- Setup: $n$ bits, insert data set $S$ into bloom filter
- Note: for each bit of $A$, the probability it's been set to $1$ is
    - $1-(1-\dfrac{1}{n})^{k|S|} \le 1 - e^{-k|S|/n} = 1 - e^{-k/b}$, $b$ = number of bits per object $\dfrac{n}{|S|}$
- Thus, false positive probability is $\le [1-e^{-k/b}]^{k} = \epsilon$ 
- For fixed $b$, $\epsilon$ is minimized by setting
    - $\epsilon \approx \dfrac{1}{2}^{(ln2)b}$

In [ ]:
import random

In [ ]:
def open_file(file_path):
    """
    Read file line by line

    Args:
    file_path (string) -- location of file to be read

    Returns:
    array (list) --  contains numbers in sorted order
    """

    array = []

    with open(file_path, 'r') as line:
        array = line.read().split("\n")

    for i in range(0, len(array)):
        array[i] = int(array[i])
        
    array.sort()    
        
    return array

In [ ]:
def two_sum(array, a,b):
    """
    Implement two sum algorithm

    Args:
    a (integer) - lower bound of a range (inclusive)
    b (integer) - upper bound of a range (inclusive)

    Returns:
    number_of_distinct_two_sum (integer) -- number of distinct x and y such that x + y = t where t is a number between a and b
    """

    low_index = 0
    high_index = len(array) - 1
    the_sum_set = set()

    # traverse towards middle from both ends
    while high_index > low_index: 
        
        # if sum is greater than b, reduce bigger number for the next iteration
        if array[low_index] + array[high_index] > b: 
            high_index -= 1
            continue
            
        # if sum is less than a, increase smaller number for the next iteration
        if array[low_index] + array[high_index] < a: 
            low_index += 1
            continue
            
        # if two numbers are not distinct, no need to check further
        if array[low_index] == array[high_index]: 
            break

        the_low_index = low_index

        # find the sum of two numbers at the current indices
        the_sum = array[the_low_index] + array[high_index] 

        # if a <= "the_sum" <= b, there is a hit
        while the_sum >= a and the_sum <= b and high_index > the_low_index: 
            the_sum_set.add(the_sum)
            the_low_index += 1
            
            # if two numbers are not distinct, no need to check further
            if array[low_index] == array[high_index]: 
                break
            the_sum = array[the_low_index] + array[high_index]
        high_index -= 1

    number_of_distinct_two_sum =len(the_sum_set)
    print(number_of_distinct_two_sum)
    return number_of_distinct_two_sum

In [ ]:
array = open_file("data/two-sum.txt")
assert(two_sum(array, -10000, 10000) == 427)